<a href="https://colab.research.google.com/github/qn19325/individualProject/blob/main/conv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms.transforms import ToTensor
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from skimage import io
import os
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data Preprocessing

In [52]:
class SpeedsAndFramesDataset(Dataset):
  def __init__(self, csv_file, root_dir, transform=None):
      self.csv_data = pd.read_csv(csv_file)
      self.root_dir = root_dir
      self.transform = transform

  def __len__(self):
      return len(self.csv_data)

  def __getitem__(self, index):
      img_path = os.path.join(self.root_dir, self.csv_data.iloc[index, 0])
      image = io.imread(img_path)
      y_label = torch.tensor(int(self.csv_data.iloc[index, 2])/10)

      if self.transform:
          image = self.transform(image)

      return (image, y_label)

In [53]:
dataset = SpeedsAndFramesDataset(
    csv_file='data.csv',
    root_dir="trainingData",
    transform=transforms.ToTensor(),
)

In [54]:
inputs = []
targets = []
for i in range(len(dataset)-10):
  pixels = []
  speeds = []
  for j in range(10):
    pixels.append(dataset[i+j][0])
    speeds.append(dataset[i+j][1])
  inputs.append(pixels)
  targets.append(speeds)

# Hyperparameters

In [56]:
input_size = 64
hidden_size = 256
num_layers = 2
num_classes = 11
sequence_length = 10
batch_size = len(inputs)
learning_rate = 0.005
num_epochs = 1

# Model

In [57]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

        out, _ = self.rnn(x, h0)
        out = out.reshape(out.shape[0], -1)

        out = self.fc(out)
        return out

# Define model

In [59]:
model = RNN(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and Optimizer

In [60]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Network

In [61]:
for epoch in range(num_epochs):
  for elem in train_loader:
      # Get data to cuda if possible
      data = data.to(device=device).squeeze(1)
      targets = targets.to(device=device)

      # forward
      scores = model(data)
      targets = targets.type(torch.LongTensor)
      loss = criterion(scores, targets)

      # backward
      optimizer.zero_grad()
      loss.backward()

      # gradient descent update step/adam step
      optimizer.step()

NameError: ignored